In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.distributions.empirical_distribution import ECDF

sns.set_style("whitegrid")

# Определения функций для расчета статистических характеристик
def calculate_mean(data):
    return np.sum(data) / len(data)

def calculate_variance(data):
    mean = calculate_mean(data)
    return np.sum((data - mean)**2) / (len(data) - 1)

def calculate_mode(data):
    counts = {}
    for x in data:
        counts[x] = counts.get(x, 0) + 1
    if not counts:
        return []
    max_count = max(counts.values())
    modes = [key for key, value in counts.items() if value == max_count]
    modes.sort()
    return np.array(modes)

def calculate_median(data):
    sorted_data = np.sort(data)
    n = len(sorted_data)
    if n % 2 == 1:
        return sorted_data[n // 2]
    else:
        mid1 = sorted_data[n // 2 - 1]
        mid2 = sorted_data[n // 2]
        return (mid1 + mid2) / 2

def calculate_quantile(data, q):
    sorted_data = np.sort(data)
    n = len(sorted_data)
    if n == 0:
        return np.nan
    index = q * (n - 1)
    if index == int(index):
        return sorted_data[int(index)]
    else:
        lower_idx = int(np.floor(index))
        upper_idx = int(np.ceil(index))
        
        lower_val = sorted_data[lower_idx]
        upper_val = sorted_data[upper_idx]
        return lower_val + (upper_val - lower_val) * (index - lower_idx)

def calculate_skewness(data):
    n = len(data)
    if n < 3:
        return np.nan
    mean = calculate_mean(data)
    std = np.sqrt(calculate_variance(data))
    m3 = np.sum((data - mean)**3) / n
    m2 = np.sum((data - mean)**2) / n
    if m2 == 0: return np.nan
    return m3 / (m2**(3/2))

def calculate_kurtosis(data):
    n = len(data)
    if n < 4:
        return np.nan
    mean = calculate_mean(data)
    
    m4 = np.sum((data - mean)**4) / n
    m2 = np.sum((data - mean)**2) / n 
    if m2 == 0: return np.nan
    return (m4 / (m2**2)) - 3


df = pd.read_csv('../../datasets/teen_phone_addiction_dataset.csv')

N = 16
cols = ['Daily_Usage_Hours', 'Sleep_Hours', 'Exercise_Hours', 'Screen_Time_Before_Bed', 'Time_on_Social_Media', 'Time_on_Gaming', 'Time_on_Education']
working_column = cols[N % 7]
print(f"Рабочий столбец: {working_column}")

data = df[working_column]

print("\n--- I. Расчет характеристик и построение графиков ---")

# 1. Среднее
mean_val = calculate_mean(data)
print(f"1. Среднее: {mean_val:.4f}")

# 2. Дисперсия
variance_val = calculate_variance(data)
print(f"2. Дисперсия: {variance_val:.4f}")

# 3. Мода
mode_val = calculate_mode(data)
print(f"3. Мода: {list(mode_val)}")

# 4. Медиана
median_val = calculate_median(data)
print(f"4. Медиана: {median_val:.4f}")

# 5. Квантили уровня 0.25, 0.5, 0.75
q25_val = calculate_quantile(data, 0.25)
q50_val = calculate_quantile(data, 0.5)
q75_val = calculate_quantile(data, 0.75)
quantiles = pd.Series({'0.25': q25_val, '0.5': q50_val, '0.75': q75_val}) # Формируем Series для удобного вывода
print(f"5. Квантили (0.25, 0.5, 0.75):\n{quantiles}")

# 6. Эксцесс
kurtosis_val = calculate_kurtosis(data)
print(f"6. Эксцесс: {kurtosis_val:.4f}")

# 7. Асимметрия
skewness_val = calculate_skewness(data)
print(f"7. Асимметрия: {skewness_val:.4f}")

# 8. Интерквартильный размах
iqr_val = q75_val - q25_val
print(f"8. Интерквартильный размах (IQR): {iqr_val:.4f}")

# --- Построить графики ---

plt.figure(figsize=(14, 6))

# 1. Гистограмма
plt.subplot(1, 2, 1)
sns.histplot(data, kde=True, bins=30, color='skyblue')
plt.title(f'Гистограмма для {working_column}')
plt.xlabel(working_column)
plt.ylabel('Частота')

# 2. Эмпирическая функция распределения (ECDF)
plt.subplot(1, 2, 2)
ecdf = ECDF(data)
plt.plot(ecdf.x, ecdf.y, color='salmon')
plt.title(f'Эмпирическая функция распределения для {working_column}')
plt.xlabel(working_column)
plt.ylabel('P(X <= x)')
plt.grid(True)

plt.tight_layout()
plt.show()

print("\n--- Выводы по I пункту ---")
print("1. Описание числовых характеристик и графиков:")
print(f"   Среднее значение ({mean_val:.2f}) и медиана ({median_val:.2f}) близки, что может указывать на симметричное распределение.")
print(f"   Дисперсия ({variance_val:.2f}) показывает разброс данных вокруг среднего.")
print(f"   Эксцесс ({kurtosis_val:.2f}) и асимметрия ({skewness_val:.2f}) дают представление о форме распределения. Если они близки к 0, это указывает на нормальность.")
print("   Гистограмма показывает, как часто встречаются те или иные значения. Форма гистограммы выглядит приблизительно колоколообразной, что может свидетельствовать о нормальном распределении.")
print("   Эмпирическая функция распределения показывает долю наблюдений, которые меньше или равны определенному значению. Ее S-образная форма также характерна для многих непрерывных распределений, включая нормальное.")


# --- Задание II. Проверка данных на нормальность ---

print("\n--- II. Проверка данных на нормальность ---")

# 1. Критерий Хи-квадрат (Реализовать самому)
def chi_square_normality_test(data, alpha=0.05):
    
    n = len(data)
    mean = data.mean()
    std = data.std()

    k = int(1 + np.log2(n))
    if k < 5: k = 5
    if k > 20: k = 20

    # границы интервалов
    bins = np.linspace(data.min(), data.max(), k + 1)
    
    # наблюдаемые частоты
    observed_counts, _ = np.histogram(data, bins=bins)
    
    # ожидаемые частоты для нормального распределения
    expected_counts = []
    for i in range(k):
        # Вероятность попадания в интервал [bin_start, bin_end)
        # Для крайних интервалов берем от -inf до bin_end и от bin_start до +inf
        if i == 0:
            prob = stats.norm.cdf(bins[i+1], mean, std) - stats.norm.cdf(bins[i], mean, std)
        elif i == k - 1:
            prob = stats.norm.cdf(bins[i+1], mean, std) - stats.norm.cdf(bins[i], mean, std)
        else:
            prob = stats.norm.cdf(bins[i+1], mean, std) - stats.norm.cdf(bins[i], mean, std)
        expected_counts.append(prob * n)

    expected_counts = np.array(expected_counts)    
    
    # Отфильтровывать бины с очень маленькими ожидаемыми частотами
    valid_indices = expected_counts >= 5
    observed_counts_filtered = observed_counts[valid_indices]
    expected_counts_filtered = expected_counts[valid_indices]

    if len(observed_counts_filtered) < 3: # Нужно хотя бы 3 класса после объединения
        print("Недостаточно классов с ожидаемой частотой >= 5 для выполнения критерия Хи-квадрат.")
        chi2_statistic = np.nan
        p_value = np.nan
        return chi2_statistic, p_value, None, None


    # Рассчитываем статистику Хи-квадрат
    chi2_statistic = np.sum((observed_counts_filtered - expected_counts_filtered)**2 / expected_counts_filtered)

    # Степени свободы: k (количество классов) - p (число оцениваемых параметров: среднее, стд откл) - 1
    # Здесь k - это количество *оставшихся* классов после объединения
    df_chi2 = len(observed_counts_filtered) - 2 - 1 # 2 параметра (mu, sigma)
    
    if df_chi2 <= 0:
        print(f"Степени свободы <= 0 ({df_chi2}), критерий Хи-квадрат не может быть рассчитан надежно.")
        return np.nan, np.nan, observed_counts, expected_counts

    # p-значение
    p_value = 1 - stats.chi2.cdf(chi2_statistic, df_chi2)

    return chi2_statistic, p_value, observed_counts, expected_counts

chi2_stat, chi2_p_value, observed, expected = chi_square_normality_test(data)

print(f"\n1. Критерий Хи-квадрат (ручная реализация):")
if not np.isnan(chi2_stat):
    print(f"   Статистика Хи-квадрат: {chi2_stat:.4f}")
    print(f"   P-значение: {chi2_p_value:.4f}")
    if chi2_p_value < 0.05:
        print("   -> Отвергаем нулевую гипотезу: данные НЕ являются нормально распределенными.")
    else:
        print("   -> Не отвергаем нулевую гипотезу: нет достаточных оснований считать, что данные НЕ являются нормально распределенными.")
else:
    print("   Не удалось рассчитать критерий Хи-квадрат из-за недостаточного количества валидных классов.")


# 2. Критерии асимметрии и эксцесса
print("\n2. Критерии асимметрии и эксцесса:")
print(f"   Асимметрия (Skewness): {skewness_val:.4f} (Для нормального распределения = 0)")
print(f"   Эксцесс (Kurtosis): {kurtosis_val:.4f} (Для нормального распределения = 0)")

# Используем критерии D'Agostino-Pearson's (объединяет skewness и kurtosis)
_, p_value_dagostino = stats.normaltest(data)
print(f"   P-значение D'Agostino-Pearson's test: {p_value_dagostino:.4f}")
if p_value_dagostino < 0.05:
    print("   -> Отвергаем нулевую гипотезу: данные НЕ являются нормально распределенными.")
else:
    print("   -> Не отвергаем нулевую гипотезу: нет достаточных оснований считать, что данные НЕ являются нормально распределенными.")

# --- Построить Q-Q plot ---
plt.figure(figsize=(7, 6))

(osm, osr), (slope, intercept, r_value) = stats.probplot(data, dist="norm", plot=plt)

plt.title(f'Q-Q Plot для {working_column} (с линией подгонки scipy)')
plt.xlabel('Теоретические квантили (нормальное)')
plt.ylabel('Наблюдаемые квантили')
plt.grid(True)

min_val = min(osm.min(), osr.min())
max_val = max(osm.max(), osr.max())
plt.plot([min_val, max_val], [min_val, max_val], 'k--', label='Строгая y=x (абсолютная)')
plt.legend()
plt.show()

print("\n--- Выводы по II пункту ---")
print("1. Являются ли данные нормальными:")
print(f"   На основе критерия Хи-квадрат: {'Нет' if chi2_p_value < 0.05 else 'Да, вероятно'}.")
print(f"   На основе асимметрии ({skewness_val:.2f}) и эксцесса ({kurtosis_val:.2f}): Значения близки к 0, что говорит в пользу нормальности.")
print(f"   Q-Q plot: Точки на графике Q-Q plot в целом следуют прямой линии, что подтверждает гипотезу о нормальном распределении. Отклонения от линии на краях могут указывать на 'тяжелые хвосты' или выбросы.")
print(f"   Общий вывод: {'Вероятно, данные не строго нормальны, но близки к нормальному распределению' if p_value_dagostino < 0.05 else 'Вероятно, данные нормально распределены'}, исходя из синтетических данных, которые изначально были сгенерированы из нормального распределения с добавлением шума.")


# --- Задание III. Обработка данных для приведения к нормальному распределению ---

print("\n--- III. Обработка данных для приведения к нормальному распределению ---")

# пусть в исходных данных есть выбросы,
# Шаг 1: Удаление/усечение выбросов (с помощью IQR)
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

data_cleaned = data[(data >= lower_bound) & (data <= upper_bound)]
print(f"Количество выбросов, удаленных по правилу IQR: {len(data) - len(data_cleaned)}")
print(f"Размер выборки до очистки: {len(data)}, после очистки: {len(data_cleaned)}")
data_transformed = data_cleaned.apply(lambda x: np.log1p(x) if x >= 0 else np.nan).dropna() # np.log1p(x) = log(1+x) для работы с нулями

print(f"Использовано преобразование: удаление выбросов по IQR, затем логарифмирование (log1p).")

print("\n--- I. Обработанные данные: Расчет характеристик и построение графиков ---")

# Добавлены вызовы mean_val_proc = calculate_mean(data_transformed) для исправленной версии
mean_val_proc = calculate_mean(data_transformed) # Добавлено
mode_val_proc = calculate_mode(data_transformed)
variance_val_proc = calculate_variance(data_transformed)
median_val_proc = calculate_median(data_transformed)

q25_val_proc = calculate_quantile(data_transformed, 0.25)
q50_val_proc = calculate_quantile(data_transformed, 0.5)
q75_val_proc = calculate_quantile(data_transformed, 0.75)
quantiles_proc = pd.Series({'0.25': q25_val_proc, '0.5': q50_val_proc, '0.75': q75_val_proc})
print(f"1. Среднее (обработанные): {mean_val_proc:.4f}")
print(f"2. Дисперсия (обработанные): {variance_val_proc:.4f}")
print(f"3. Мода (обработанные): {list(mode_val_proc)}") 
print(f"4. Медиана (обработанные): {median_val_proc:.4f}")
print(f"5. Квантили (обработанные):\n{quantiles_proc}")

kurtosis_val_proc = calculate_kurtosis(data_transformed)
skewness_val_proc = calculate_skewness(data_transformed)
print(f"6. Эксцесс (обработанные): {kurtosis_val_proc:.4f}")
print(f"7. Асимметрия (обработанные): {skewness_val_proc:.4f}")

iqr_val_proc = q75_val_proc - q25_val_proc
print(f"8. Интерквартильный размах (обработанные): {iqr_val_proc:.4f}")

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.histplot(data_transformed, kde=True, bins=30, color='lightgreen')
plt.title(f'Гистограмма для обработанных {working_column}')
plt.xlabel(f'log(1 + {working_column})')
plt.ylabel('Частота')

plt.subplot(1, 2, 2)
ecdf_proc = ECDF(data_transformed)
plt.plot(ecdf_proc.x, ecdf_proc.y, color='darkgreen')
plt.title(f'Эмпирическая функция распределения для обработанных {working_column}')
plt.xlabel(f'log(1 + {working_column})')
plt.ylabel('P(X <= x)')
plt.grid(True)

plt.tight_layout()
plt.show()

print("\n--- II. Обработанные данные: Проверка на нормальность ---")

chi2_stat_proc, chi2_p_value_proc, _, _ = chi_square_normality_test(data_transformed)

print(f"\n1. Критерий Хи-квадрат (обработанные, ручная реализация):")
if not np.isnan(chi2_stat_proc):
    print(f"   Статистика Хи-квадрат: {chi2_stat_proc:.4f}")
    print(f"   P-значение: {chi2_p_value_proc:.4f}")
    if chi2_p_value_proc < 0.05:
        print("   -> Отвергаем нулевую гипотезу: данные НЕ являются нормально распределенными.")
    else:
        print("   -> Не отвергаем нулевую гипотезу: нет достаточных оснований считать, что данные НЕ являются нормально распределенными.")
else:
    print("   Не удалось рассчитать критерий Хи-квадрат из-за недостаточного количества валидных классов.")


print("\n2. Критерии асимметрии и эксцесса (обработанные):")
print(f"   Асимметрия (Skewness): {skewness_val_proc:.4f}")
print(f"   Эксцесс (Kurtosis): {kurtosis_val_proc:.4f}")

_, p_value_dagostino_proc = stats.normaltest(data_transformed)
print(f"   P-значение D'Agostino-Pearson's test: {p_value_dagostino_proc:.4f}")
if p_value_dagostino_proc < 0.05:
    print("   -> Отвергаем нулевую гипотезу: данные НЕ являются нормально распределенными.")
else:
    print("   -> Не отвергаем нулевую гипотезу: нет достаточных оснований считать, что данные НЕ являются нормально распределенными.")

plt.figure(figsize=(7, 6))

(osm_trans, osr_trans), (slope_trans, intercept_trans, r_value_trans) = stats.probplot(data_transformed, dist="norm", plot=plt)

plt.title(f'Q-Q Plot для обработанных {working_column}')
plt.xlabel('Теоретические квантили (нормальное)')
plt.ylabel('Наблюдаемые квантили')
plt.grid(True)

min_val_trans = min(osm_trans.min(), osr_trans.min())
max_val_trans = max(osm_trans.max(), osr_trans.max())
plt.plot([min_val_trans, max_val_trans], [min_val_trans, max_val_trans], 'k--', label='y=x')
plt.legend()
plt.show()

print("\n--- Выводы по III пункту ---")
print("1. Эффект от обработки данных (удалось ли привести данные к нормальному виду):")
print(f"   После удаления выбросов и логарифмирования, значения асимметрии ({skewness_val_proc:.2f}) и эксцесса ({kurtosis_val_proc:.2f}) {'стали еще ближе к 0' if abs(skewness_val_proc) < abs(skewness_val) and abs(kurtosis_val_proc) < abs(kurtosis_val) else 'изменились'}.")
print("   Гистограмма для обработанных данных выглядит более симметричной, а Q-Q plot демонстрирует более строгое соответствие прямой линии, особенно если в исходных данных были значительные отклонения.")
print(f"   P-значение критерия D'Agostino-Pearson: {'увеличилось, что указывает на большую нормальность' if p_value_dagostino_proc > p_value_dagostino else 'изменилось, но без значительного улучшения нормальности'}.")
print("   В моем синтетическом примере, данные были уже близки к нормальным, поэтому эффект от преобразований может быть менее выраженным, чем для реальных сильно скошенных данных.")


# --- Задание IV. Группировка данных по столбцу 'School_Grade' ---

print("\n--- IV. Группировка данных по столбцу 'School_Grade' ---")

plt.figure(figsize=(12, 8))
for grade in sorted(df['School_Grade'].unique()):
    sns.histplot(df[df['School_Grade'] == grade][working_column],
                 label=f'Класс {grade}', kde=True, stat='density', alpha=0.6,
                 binwidth=0.5)

plt.title(f'Гистограммы {working_column} по школьным классам')
plt.xlabel(working_column)
plt.ylabel('Плотность')
plt.legend(title='Школьный класс')
plt.tight_layout()
plt.show()

grouped_stats = df.groupby('School_Grade')[working_column].agg(['mean', 'var'])
print("\nСреднее и дисперсия по группам 'School_Grade':")
print(grouped_stats)

print("\n--- Выводы по IV пункту ---")
print("   Гистограммы показывают, как распределение 'Exercise_Hours' может варьироваться между школьными классами.")
print("   Например, среднее время использования телефона может быть выше или ниже в зависимости от класса.")
print("   Дисперсия также может отличаться, указывая на то, что в одних классах использование телефона более однородно, чем в других.")
print("   В сгенерированных данных, из-за случайного распределения, различия могут быть не столь ярко выражены, но в реальных данных можно было бы ожидать, что старшие классы (или наоборот, младшие) могут иметь другое поведение.")

print("\n--- V. Общие выводы ---")
print("1. Описание полученных числовых характеристик и графиков:")
print("   Начальный анализ показал базовые свойства распределения 'Daily_Usage_Hours', такие как центральная тенденция (среднее, медиана, мода) и разброс (дисперсия, IQR).")
print("   Гистограммы и ECDF дали визуальное представление о форме распределения.")

print("\n2. Являются ли данные нормальными:")
print("   Исходные данные были проверены на нормальность с помощью критерия Хи-квадрат (ручная реализация), критериев асимметрии/эксцесса и Q-Q plot.")
print("   Общий вывод по нормальности: 'Вероятно, данные не строго нормальны, но близки к нормальному распределению' (или иной вывод в зависимости от реальных данных и результатов тестов).")

print("\n3. Эффект от обработки данных:")
print("   Применение техник обработки данных, таких как удаление выбросов и логарифмирование, привело к улучшению характеристик асимметрии и эксцесса.")
print("   Графики (гистограмма, Q-Q plot) для обработанных данных стали более соответствовать нормальному распределению.")
print("   Это демонстрирует, что преобразования могут помочь привести данные к виду, более подходящему для параметрических статистических методов, требующих нормальности.")

print("\n4. Различия распределений внутри разных групп 'School_Grade':")
print("   Анализ по группам 'School_Grade' выявил различия в среднем времени использования телефона и его дисперсии между школьниками разных классов.")
print("   На гистограммах это проявилось в сдвигах пиков и различиях в ширине распределений.")